In [21]:
#Sequential 모댈을 만들고 모델에 필요한 레이어를 추가하는 방법

In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense

In [23]:
# 컬럼명 정의 (마지막이 레이블 target)
cols = [
    "area", "perimeter", "compactness",
    "length", "width", "asymmetry", "groove_length",
    "target"
]

# 1) 데이터 로드 (확장자 없으면 그냥 경로, sep는 공백 기준)
df = pd.read_csv('./seeds_dataset.txt', sep=r"\s+", names=cols)

In [24]:
# 2. 특성과 레이블 분리
X = df.drop("target", axis=1).values      # shape: (210, 7)
y = df["target"].values - 1               # 원래 레이블은 1,2,3 → 0,1,2로 변환

In [25]:
# 3. train/test 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [26]:
# 4. 스케일링 (필수는 아니지만 안정적인 학습을 위해 권장)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [27]:
# 5. 텐서로 변환
#    (tf.data.Dataset을 써도 되지만, 여기서는 간단히 넘파이를 바로 Keras에 넘깁니다)
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)
X_test  = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test  = tf.convert_to_tensor(y_test, dtype=tf.int32)

In [28]:
# 6) 모델 정의
model = Sequential()
# - 은닉층: 유닛 16, 입력 차원 7
model.add(Dense(units=16, input_shape=(7,), activation='relu'))
# - 추가 은닉층: 유닛 8
model.add(Dense(units=8, activation='relu'))
# - 출력층: 클래스 수 3 → units=3, softmax
model.add(Dense(units=3, activation='softmax'))

C:\Users\leehy\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
# 7) 컴파일
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 8) 학습
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=16,
    verbose=1
)

Epoch 1/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 0.2178 - loss: 1.3229 - val_accuracy: 0.3824 - val_loss: 1.1958
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.3265 - loss: 1.1533 - val_accuracy: 0.5000 - val_loss: 1.0725
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5349 - loss: 1.0489 - val_accuracy: 0.5882 - val_loss: 0.9568
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6565 - loss: 0.9400 - val_accuracy: 0.7353 - val_loss: 0.8528
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7388 - loss: 0.8377 - val_accuracy: 0.7647 - val_loss: 0.7625
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7814 - loss: 0.7958 - val_accuracy: 0.7941 - val_loss: 0.6915
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8266 - loss: 0.7344 - val_accuracy: 0.8235 - val_loss: 0.6331
Epoch 8/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8430 - loss: 0.6637 - val_accuracy: 0.8235 - val_loss: 0.5843


In [30]:
# 9. 평가
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"테스트 데이터 정확도: {acc:.4f}")


테스트 데이터 정확도: 0.8810
